# Density-based spatial clustering of applications with noise (DBSCAN)

---

DBSCAN is a clustering algorithm that groups datapoints together based on "density". Nearby points get assigned to a common cluster, and outlier points get assigned to their own clusters. DBSCAN is effective and attractive for its simplicity and minimal pre-specified parameters.

There are only two parameters that need to be specified for DBSCAN:

    eps : a minimum distance between points that can define a "connection"
    
    min_samples : minimum number of points that a point needs to have 
                  as neighbors to define it as a "core sample"
    
**Core samples** are by design the points that lie internally within a cluster. 

**Non-core samples** do not meet the minimum required neighboring points, but are still connected to a cluster defined by a core sample or samples. Hence these points lie on the edges of a cluster.

**Outliers** are points that do not meet the distance criteria to a cluster nor minimum neighbors to form a new cluster.


---

## DBSCAN algorithm

The DBSCAN algorithm proceeds iteratively through the points, determining via the distance measure and minimum samples specified whether points are core samples, edge samples, or outliers.

Here is the pseudocode algorithm below, which we will be coding up ourselves:


```
DBSCAN(D, eps, MinPts) {
   C = 0
   for each point P in dataset D {
      if P is visited
         continue next point
      mark P as visited
      NeighborPts = regionQuery(P, eps)
      if sizeof(NeighborPts) < MinPts
         mark P as NOISE
      else {
         C = next cluster
         expandCluster(P, NeighborPts, C, eps, MinPts)
      }
   }
}

expandCluster(P, NeighborPts, C, eps, MinPts) {
   add P to cluster C
   for each point P' in NeighborPts { 
      if P' is not visited {
         mark P' as visited
         NeighborPts' = regionQuery(P', eps)
         if sizeof(NeighborPts') >= MinPts
            NeighborPts = NeighborPts joined with NeighborPts'
      }
      if P' is not yet member of any cluster
         add P' to cluster C
   }
}

regionQuery(P, eps)
   return all points within P's eps-neighborhood (including P)
```

---

## DBSCAN in parts

We can roll our own DBSCAN following the pseudcode above. Doing it piece by piece in parts will make it clear how the algorithm works.

### 1. Create some clustered data

sklearn has some nice data generation functions in its `sklearn.datasets` module. I've loaded a handful of them below. You can use them to create clustered data easily to test clustering algorithms.

Generate clustered data and plot it out.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# Dataset generators:
from sklearn.datasets import make_biclusters, make_blobs, make_circles, make_moons

---

### 2. Make the skeleton of the DBSCAN class

Start laying out the blueprint for how DBSCAN will work. We'll need to start out:

1. An `__init__` function to initialize the class with the `eps` and `min_samples` arguments.
- A (for now empty) `fit` function that will run DBSCAN on the data.

---

### 3. Writing the `fit` function (equivalent to the `DBSCAN` function in pseudocode)

Our `fit` function will follow the logic of the `DBSCAN` function in the pseudocode above (re-pasted here). In general, when building classes, think about what variables are best suited to be class attributes. It takes practice to get a feel for it.

```
DBSCAN(D, eps, MinPts) {
   C = 0
   for each point P in dataset D {
      if P is visited
         continue next point
      mark P as visited
      NeighborPts = regionQuery(P, eps)
      if sizeof(NeighborPts) < MinPts
         mark P as NOISE
      else {
         C = next cluster
         expandCluster(P, NeighborPts, C, eps, MinPts)
      }
   }
}
```

---

### 4. Write the function to find neighbors

We need to convert this function in the pseudocode to a class function:

```
regionQuery(P, eps)
   return all points within P's eps-neighborhood (including P)
```

I've already named this `self.find_region_points(i)` so far, so we'll call it that.

---

### 5. Write the function to expand the clusters.

The final function (and the one that actually assigns our clusters) is defined by the pseudocode as:

```
expandCluster(P, NeighborPts, C, eps, MinPts) {
   add P to cluster C
   for each point P' in NeighborPts { 
      if P' is not visited {
         mark P' as visited
         NeighborPts' = regionQuery(P', eps)
         if sizeof(NeighborPts') >= MinPts
            NeighborPts = NeighborPts joined with NeighborPts'
      }
      if P' is not yet member of any cluster
         add P' to cluster C
   }
}
```

Essentially the function takes a point id, the neighboring point ids, the cluster number, minimum distance, and minimum points, and figures out based on those components what cluster a point should be in. 

I've already pre-named this function in the `fit` function to be `expand_cluster(i, neighbors)`. We only need to pass in the current point and neighboring points because we're storing all of the other information the function needs as class attributes.

---

## Plotting DBSCAN interactively

We can look at how the `eps` and `min_samples` parameters affect DBSCAN's decisions about assigning clusters. This is also a good opportunity to go over how to make interactive visualizations with ipython widgets.

### 1. Re-write the plotting function to accept cluster labels and color the points accordingly

---

### 2. Write a function that accepts `eps` and `min_samples` as keyword arguments, fits DBSCAN, and calls the plotting function

Don't pass `X` in to the function. We will just use the "global" X defined in the jupyter notebook earlier.

---

### 3. Make the "interact" function that creates the ipython widget

Your interact function defines the sliders (or other inputs you want to be interactive) and then makes a call to `widgets.interact(function, *interactive objects)` where:

- "function" is the function that takes the values coming out of the interactive slider objects (we wrote it just before: it takes `eps` and `min_samples`
- The interactive objects in our case will be a `widgets.FloatSlider` for eps and a `widgets.IntSlider` for the min_samples.

For more information see this handy notebook:

https://github.com/ipython/ipywidgets/blob/master/docs/source/examples/Index.ipynb